<a href="https://colab.research.google.com/github/uzdanska/Machine-Learning/blob/main/supervised/01_basics/01_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### scikit-learn

Strona biblioteki: https://scikit-learn.org

Dokumentacja/User Guide: https://scikit-learn.org/stable/user_guide.html

Podstawowa biblioteka do uczenia maszynowego w języku Python.

Aby zainstalować bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install scikit-learn

```
Aby zaktualizować do najnowszej wersji bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install --upgrade scikit-learn
```
Kurs stworzony w oparciu o wersję 0.22.1

### Preprocessing danych:


1.   [Import bibliorek](#0)
2.   [Wygenerowanie danych](#1)
3.   [Utworzenie kopii danych](#2)
4.   [Zmiana typu danych i wstępna eksploracja](#3)
5.   [Przygotowanie obiektu dataframe do uczenia maszynowego](#4)

    5.1.  [Zmiana target (**'bought'**) na wartość numeryczną](#4_1)

    5.2. [Zmiana zmiennych opisujących (**data**) na wartości numeryczne](#4_2)

    5.3   [Pandas get_dummies()](#4_3)
6.   [Standaryzacja zmiennych numerycznych](#5)

    6.1. [Średnia i odchylenie standardowe w bibliotece](#5_1)

    6.2. [Gotowa Standaryzacja z sklearn **scale**](#5_2)

    6.3 [Klasa StandardScaler](#5_3)
7.   [Przygotowanie danych podsumowanie](#6)

### <a name="0"></a> 1. Import bibliotek

In [44]:
import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.2.2'

### <a name="1"></a> 2. Wygenerowanie danych

In [84]:
data = {
    'size': ['XL', 'L', 'M', 'S', 'XS'],
    'color': ['red', 'blue', 'blue', 'white', 'red'],
    'gender': ['female', 'male', 'male', 'female', 'female'],
    'price': [199.0, 89.0, 109.0, 79.0, 149.0],
    'weight': [500, 450, 300, 400, 385],
    'bought': ['yes', 'no', 'no', 'yes', 'yes']
}

df_raw = pd.DataFrame(data = data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,blue,male,89.0,450,no
2,M,blue,male,109.0,300,no
3,S,white,female,79.0,400,yes
4,XS,red,female,149.0,385,yes


### <a name="2"></a> 3. Utworzenie kopii danych

In [25]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


### <a name="3"></a> 4. Zmiana typu danych i wstępna eksploracja

In [26]:
# zmiana typu object na category
for col in ['size', 'color', 'gender', 'bought']:
  df[col] = df[col].astype('category')

df['weight'] = df['weight'].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 820.0 bytes


In [27]:
# podstawowe statystyki
# domyslnie ta metoda wyswietla statystyki tylko dla wartosci numerycznych temu
# mamy tylko 2 kolumny float64
df.describe()

,price,weight
count,5.00000,5.000000
mean,125.00000,407.000000
std,49.29503,74.966659
min,79.00000,300.000000
25%,89.00000,385.000000
50%,109.00000,400.000000
75%,149.00000,450.000000
max,199.00000,500.000000


In [28]:
# metoda T transponuje nam macierz dzięki, czemu jest łatwiejsza do odczytu
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,5.0,125.0,49.295030,79.0,89.0,109.0,149.0,199.0
weight,5.0,407.0,74.966659,300.0,385.0,400.0,450.0,500.0


In [29]:
# Wyswietlenie rowniez statystyk zmiennych nienumerycznych czyli category
df.describe(include = ['category']).T

# count liczba elementow ktore wystepuja
# unique liczba unikalnych wartosci
# top wartosc ktora pojawia się najcześciej
# freq frewfekcja czyli częstośc występowiania wartości

,count,unique,top,freq
size,5,5,L,1
color,5,3,blue,2
gender,5,2,female,3
bought,5,2,yes,3


### <a name="4"></a> 5.Przygotowanie obiektu dataframe do uczenia maszynowego

Poniżej znajduje się podgląd modelu:

In [30]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,blue,male,89.0,450.0,no
2,M,blue,male,109.0,300.0,no
3,S,white,female,79.0,400.0,yes
4,XS,red,female,149.0,385.0,yes


Poniewaz w większości dane przekazywanie do uczenia maszynowego są danymi numerycznymi, to musimy je uporządkować zrobimy to przy wykorzystaniu **LabelEncoder** ktora jest przystowana głównie do zmiennej docelowej (target).
Do zmiennych opisujących (data) mamy inne narzędzie.

Oto kilka kroków:

#### <a name="4_1"></a> 5.1. Zmiana target (**'bought'**) na wartość numeryczną:


Metody **LabelEncoder** służące do zmiany wartości docelowej na zmienna numeryczną to:

*  **fit** dopasowuje nasz encoder do podanej w zmiennej,

* **transform** przetransformuje podane wartości na dane numeryczne czyli 0 i 1.

In [31]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df['bought'])
le.transform(df['bought'])


array([1, 0, 0, 1, 1])

Możemy też połaczyć metody **fit** i **transform** w jedną **fit_transform**:

In [32]:
le.fit_transform(df['bought'])

array([1, 0, 0, 1, 1])

Wyświetlenie mapowania encodera zmiennej:

*   0 == 'no'
*   1 == 'yes'



In [33]:
le.classes_

array(['no', 'yes'], dtype=object)

Przypisanie mapowania do tabeli DataFrame

In [34]:
df['bought'] = le.fit_transform(df['bought'])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,blue,male,89.0,450.0,0
2,M,blue,male,109.0,300.0,0
3,S,white,female,79.0,400.0,1
4,XS,red,female,149.0,385.0,1


Wrócenie do pierwotnej postaci DataFrame i przypisanie jej do DataFrame

In [35]:
le.inverse_transform(df['bought'])

array(['yes', 'no', 'no', 'yes', 'yes'], dtype=object)

In [36]:
df['bought'] = le.inverse_transform(df['bought'])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,blue,male,89.0,450.0,no
2,M,blue,male,109.0,300.0,no
3,S,white,female,79.0,400.0,yes
4,XS,red,female,149.0,385.0,yes


#### <a name="4_2"></a> 5.2. Zmiana zmiennych opisujących (**data**) na wartości numeryczne



Wartość 'sparse' jest ustawiona na False, ponieważ domyślnie ta klasa zwraca nam macierz rzadką której nie jesteśmy w stanie jej wyświetlić.

Macierz rzadka - to macierz która której większość wartości to 0. Dlatego, aby zwlonić pamięć przetrzymuje ona pozycje tylko gdzie jest 1. (gdy sprase = True)

In [38]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
encoder.fit(df[['size']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(sparse=False, sparse_output=False)

In [39]:
encoder.transform(df[['size']])

array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.]])

Jest to macierz numpy array 5 x 5 tam gdzie w naszych danych pojawia się ta wartosć mamy 1, kolejnośc można sprawdzić używajać ```encode.categories_```

In [40]:
encoder.categories_

[array(['L', 'M', 'S', 'XL', 'XS'], dtype=object)]

Aby unikać pomyłki można przekazać w metodzie OneHotEncoder drop='first' usunie on pierwszą z tej zmiennej.

In [41]:

encoder = OneHotEncoder(drop="first", sparse=False)
encoder.fit(df[['size']])
encoder.transform(df[['size']])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

In [42]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,blue,male,89.0,450.0,no
2,M,blue,male,109.0,300.0,no
3,S,white,female,79.0,400.0,yes
4,XS,red,female,149.0,385.0,yes


#### <a name="4_3"></a> 5.3. Pandas get_dummies


get_dummies() to metoda z biblioteki pandas która bardzo jest bardziej elastyczna niż OneHotEncoder z punktu 5.2. Zmienia ona wszystkie wartości nienumerycznej na wartości:

  * 0 - niewystępują,
  * 1- występuje.


In [47]:
pd.get_dummies(data=df)

,price,weight,size_L,size_M,size_S,size_XL,size_XS,color_blue,color_red,color_white,gender_female,gender_male,bought_no,bought_yes
0,199.0,500.0,0,0,0,1,0,0,1,0,1,0,0,1
1,89.0,450.0,1,0,0,0,0,1,0,0,0,1,1,0
2,109.0,300.0,0,1,0,0,0,1,0,0,0,1,1,0
3,79.0,400.0,0,0,1,0,0,0,0,1,1,0,0,1
4,149.0,385.0,0,0,0,0,1,0,1,0,1,0,0,1


**drop_first=True** usuwa pierwszą kolumnę każdej zmiennej nienumerycznej

In [48]:
pd.get_dummies(data=df, drop_first=True)

,price,weight,size_M,size_S,size_XL,size_XS,color_red,color_white,gender_male,bought_yes
0,199.0,500.0,0,0,1,0,1,0,0,1
1,89.0,450.0,0,0,0,0,0,0,1,0
2,109.0,300.0,1,0,0,0,0,0,1,0
3,79.0,400.0,0,1,0,0,0,1,0,1
4,149.0,385.0,0,0,0,1,1,0,0,1


**prefix** do zmienianych danych kategorycznych dodaje do nazwy kolumny prefix np: "new"

In [50]:
pd.get_dummies(data=df, drop_first=True, prefix="new")

,price,weight,new_M,new_S,new_XL,new_XS,new_red,new_white,new_male,new_yes
0,199.0,500.0,0,0,1,0,1,0,0,1
1,89.0,450.0,0,0,0,0,0,0,1,0
2,109.0,300.0,1,0,0,0,0,0,1,0
3,79.0,400.0,0,1,0,0,0,1,0,1
4,149.0,385.0,0,0,0,1,1,0,0,1


**prefix_sep** ustawia prefix seperacyjnych zmienianych danych kategorycznych

In [51]:
# ustawienie separatora danych kategorycznych
pd.get_dummies(data=df, drop_first=True, prefix_sep="-")

,price,weight,size-M,size-S,size-XL,size-XS,color-red,color-white,gender-male,bought-yes
0,199.0,500.0,0,0,1,0,1,0,0,1
1,89.0,450.0,0,0,0,0,0,0,1,0
2,109.0,300.0,1,0,0,0,0,0,1,0
3,79.0,400.0,0,1,0,0,0,1,0,1
4,149.0,385.0,0,0,0,1,1,0,0,1


**columns** stosuje kodowanie czyli zmianę danych kategorycznych do wybranej kolumny

In [52]:
pd.get_dummies(data=df, drop_first=True, columns=['size'])

,color,gender,price,weight,bought,size_M,size_S,size_XL,size_XS
0,red,female,199.0,500.0,yes,0,0,1,0
1,blue,male,89.0,450.0,no,0,0,0,0
2,blue,male,109.0,300.0,no,1,0,0,0
3,white,female,79.0,400.0,yes,0,1,0,0
4,red,female,149.0,385.0,yes,0,0,0,1


### <a name="5"></a> 6. Standaryzacja zmiennych numerycznych

Zmienne numeryczne:

  * 'price'
  * 'weight'

Doprowadzić do takiej postaci w której skala wartości nie ma znaczenia, natomiast znaczenie ma ich rorzut (wariancja).

W bibliotece sklearn mamy zmienna do standaryzacji StandardScaler.

Odchylenie standardowe jest estymatorem obciążonym wariancji- POCZYTAJ.

std() - pandas nieobciązony

std() - numpy obciążony

#### <a name="5_1"></a> 6.1. Średnia i odchylenie standardowe w bibliotece Pandas

In [53]:
print(f"{df['price']}")
print(f"Średnia: {df['price'].mean()}")
print(f"Odchylenie standardowe: {df['price'].std()}")

0    199.0
1     89.0
2    109.0
3     79.0
4    149.0
Name: price, dtype: float64
Średnia: 125.0
Odchylenie standardowe: 49.29503017546495


Standaryzacja price

In [54]:
(df['price'] - df['price'].mean()) / df['price'].std()

0    1.501166
1   -0.730297
2   -0.324576
3   -0.933157
4    0.486864
Name: price, dtype: float64

In [55]:
def standardize(x):
  return (x - x.mean()) / x.std()

standardize(df['price'])

0    1.501166
1   -0.730297
2   -0.324576
3   -0.933157
4    0.486864
Name: price, dtype: float64

#### <a name="5_2"></a> 6.2. Gotowa Standaryzacja z sklearn **scale**

Tutaj wartości nieco się różnią od tych przy użyciu pandas, jest to kwestia estymatora obciążonego i nieobciążonego wariancji.

Wartość odchylenia standardowego też ulegnie zmianie

In [56]:
from sklearn.preprocessing import scale

scale(df['price'])


array([ 1.67835408, -0.81649658, -0.36288737, -1.04330119,  0.54433105])

#### <a name="5_3"></a> 6.3. Klasa StandardScaler

jest ona nieco lepsza od metody scale, ze względu na to, że klasa pozwala zachować nam na przyszłość wartości statystyk. Pozwalają tym samym na łatwiejsze przetwarzanie danych, jeśli takie się pojawią.

In [72]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df[['price']])
scaler.transform(df[['price']])

array([[ 1.67835408],
       [-0.81649658],
       [-0.36288737],
       [-1.04330119],
       [ 0.54433105]])

#### <a name="5_3"></a> 6.4 Zapis do modelu wszystkich wystandaryzowanych wartości

In [60]:
scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.678354,1.386979,yes
1,L,blue,male,-0.816497,0.641291,no
2,M,blue,male,-0.362887,-1.595771,no
3,S,white,female,-1.043301,-0.104396,yes
4,XS,red,female,0.544331,-0.328102,yes


### <a name="6"></a> 7. Przygotowanie danych podsumowanie

In [76]:
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,blue,male,89.0,450,no
2,M,blue,male,109.0,300,no
3,S,white,female,79.0,400,yes
4,XS,red,female,149.0,385,yes


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


In [78]:
for col in ['size', 'color', 'gender', 'bought']:
  df[col] = df[col].astype('category')

df['weight'] = df['weight'].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 820.0 bytes


In [81]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df['bought'] = labelencoder.fit_transform(df['bought'])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,blue,male,89.0,450.0,0
2,M,blue,male,109.0,300.0,0
3,S,white,female,79.0,400.0,1
4,XS,red,female,149.0,385.0,1


In [82]:
df = pd.get_dummies(data=df, drop_first=True)
df

,price,weight,bought,size_M,size_S,size_XL,size_XS,color_red,color_white,gender_male
0,199.0,500.0,1,0,0,1,0,1,0,0
1,89.0,450.0,0,0,0,0,0,0,0,1
2,109.0,300.0,0,1,0,0,0,0,0,1
3,79.0,400.0,1,0,1,0,0,0,1,0
4,149.0,385.0,1,0,0,0,1,1,0,0


In [83]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['price', 'weight']] = scaler.fit_transform(df[['price', 'weight']])
df

,price,weight,bought,size_M,size_S,size_XL,size_XS,color_red,color_white,gender_male
0,1.678354,1.386979,1,0,0,1,0,1,0,0
1,-0.816497,0.641291,0,0,0,0,0,0,0,1
2,-0.362887,-1.595771,0,1,0,0,0,0,0,1
3,-1.043301,-0.104396,1,0,1,0,0,0,1,0
4,0.544331,-0.328102,1,0,0,0,1,1,0,0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')